### 基礎框架
##### 練習抓取ptt電影版的網站資料
##### 可以用F12開啟網站的開發者工具，查看網站的資料、找出需要的資訊，這次的在network-->index.html--> request headers

In [1]:
import urllib.request as req
import bs4
import json
from pathlib import Path
base_path = Path(r'%notebook').absolute().parent.parent.parent
with open(fr"{base_path}\config.json", "r") as f:
    config = json.load(f)

In [2]:
url = "https://www.ptt.cc/bbs/movie/index.html"
# 建立一個request物件，附加request header的資訊
request = req.Request(url, headers={
    'User-Agent':config["request_headers"]["User-Agent"]
}
)
with req.urlopen(request) as response:
    data = response.read().decode('utf-8')

In [3]:
#解析原始碼，獲得每篇文章的標題
root = bs4.BeautifulSoup(data, "html.parser")
titles = root.find_all("div", class_="title" ) # 尋找所有class = 'title'的div標籤
for title in titles:
    if title.a != None:
        print(title.a.string)

[好雷]《你傷了我的心》，比起善意鼓勵，更需要
[情報] 《寬鬆世代又怎樣》上映戲院列表
[  普偏好雷] 金髮夢露
Re: [片單] 其他電影角色出現在電影裡
Re: [負雷] 心得 NETFLIX新片《斷訊》真想斷訊不看
[情報] 阿比查邦將和蒂妲史雲頓再度合作
[公告] 電影板板規 2022/12/5
[公告] 禁政治版規 及 投票結果


### Cookie 操作實務
##### 連續抓取頁面，把所有頁面的資料都抓出來
##### 以ptt八卦版的網頁為例，但是八卦版會跳出訊息詢問使否滿18歲，且會透過cookie紀錄，所以爬蟲需要加上滿18歲的cookie，才能順利爬到資料
##### cookie可以F12 --> Application --> Storage ---> Cookies 裡面觀察
##### 觀察完之後可以再network-->index.html--> request headers裡面看怎麼顯示

In [4]:
def get_title_data(url, headers):
    # 建立一個request物件，附加request header的資訊
    request = req.Request(url, headers = headers)
    with req.urlopen(request) as response:
        data = response.read().decode('utf-8')

    #解析原始碼，獲得每篇文章的標題
    root = bs4.BeautifulSoup(data, "html.parser")
    titles = root.find_all("div", class_="title" ) # 尋找所有class = 'title'的div標籤
    for title in titles:
        if title.a != None:
            print(title.a.string)

    # 要找出能夠到上一頁超連結網址，透過包含"上頁"字段的a標籤來找
    nextlink = root.find("a", string="‹ 上頁")
    return nextlink["href"]

In [5]:
page_url = "https://www.ptt.cc/bbs/Gossiping/index.html"
page_headers={
    'User-Agent':config["request_headers"]["User-Agent"],
    'cookie':'over18=1'
}

count =0
while count <3:
    page_url = "https://www.ptt.cc" + get_title_data(page_url, page_headers)
    count +=1

[問卦] 逃逸外勞生的小孩 是無國籍人球嗎?
Re: [問卦] 資產落後的嚴重焦慮如何解
[公告] 八卦板板規(2023.11.11)
[公告] 暫時性板規 期限至2024年01月20日
[協尋] 家中失智長輩走失 12/6 台北市大安區
[協尋] 超商遇色老闆 親身體驗
[公告] hololive相關幻想文請至該專板發表
[問卦] 慟！托椅子今年最後一次直播
[問卦] 沒有方文山周杰倫還會那麼紅嗎？
[問卦] 如果拿到全部的無限寶石 會想做什麼?
[新聞] 不會當選的人 說什麼蔣經國呢
[問卦] 你各位精神故鄉是哪裡
[問卦] 台男平均長度到底幾公分
[問卦] 阿傑會不會被假冒?
[問卦] 今天台南的雙主菜便當菜色如何？
[新聞] 柯文哲稱內閣制推動困難　趙少康：奇怪
[新聞] 美國桂格燕麥疑遭沙門氏菌汙染 食藥署：
Re: [問卦] 以巴戰爭 現在風向要站哪邊?
Re: [問卦] 資產落後的嚴重焦慮如何解
[問卦] 血多40%他媽到底怎麼打啦==
[新聞] 凍漲近30年 杜微稱票價擬調漲？台鐵局：
[問卦] 抽水馬達開開關關究竟該怎麼解決呢
[新聞] 快訊／賴清德：中國的大運已經走完了 香
Re: [問卦] 資產落後的嚴重焦慮如何解
[問卦] 明年1/13是我生日
[問卦] 通膨是不是慢慢消滅台灣的人情味？
Re: [問卦] 明年1月13，八卦版全面崩潰機率多高？
[問卦] 台灣歌手為何只會唱沒營養芭樂歌
Re: [問卦] 明年1月13，八卦版全面崩潰機率多高？
[新聞] 侯友宜怒嗆「只有我槍戰過」 批賴騙義
Re: [問卦] 海軍在船上幾個月怎麼解決性需求啊？
[問卦] 孟姜女哭倒萬里長城才2次 違建要哭幾次?
Re: [問卦] 為啥愛莉莎莎以前在八卦被噓爆現在推爆
[新聞] 就是不讓！2女「狹路相逢」僵持1個多小
[問卦] 烏俄戰爭最新情報（更新至20231216）
[問卦] 對北京上海來說 台北是鄉下吧？
[問卦] 怎樣的屁股可以吃那個瀉藥
[問卦] 虛擬幣選舉賭盤怎麼抓的啊
[問卦] 現在智力測驗做到哪了？
[問卦] 開車沒禮讓肥宅過馬路是否該加倍罰
[問卦] 幹你娘真的老了現在只吃得完一隻雞
Re: [新聞] 桃園造勢也飆破萬人　柯文哲感動：沒法像
[問卦] "屠龍勇者變成惡龍"典故哪來的？
[問卦] 長輩看的電視要FHD還是4K?
[問卦] 阿傑小跟